In [1]:
!apt-get update -qq
!apt-get install -y openjdk-11-jdk-headless

!wget -q https://archive.apache.org/dist/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz
!tar -xf spark-3.5.0-bin-hadoop3.tgz


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  openjdk-11-jre-headless
Suggested packages:
  openjdk-11-demo openjdk-11-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  | fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-11-jdk-headless openjdk-11-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 6 not upgraded.
Need to get 116 MB of archives.
After this operation, 258 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 openjdk-11-jre-headless amd64 11.0.29+7-1ubuntu1~22.04 [42.6 MB]
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 

In [2]:
!pip install -q pyspark findspark


In [3]:
import os, findspark

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.0-bin-hadoop3"
os.environ["PYSPARK_PYTHON"] = "/usr/bin/python3"

findspark.init()


In [4]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("FIFA Spark SQL Lab") \
    .getOrCreate()

print("Spark version :", spark.version)


Spark version : 3.5.0


In [5]:
from google.colab import files
uploaded = files.upload()


Saving fifaworldcup.csv to fifaworldcup.csv


In [7]:
df = spark.read.option("header", True).option("inferSchema", True).csv("/content/fifaworldcup.csv")
df.printSchema()

root
 |-- date: date (nullable = true)
 |-- home_team: string (nullable = true)
 |-- away_team: string (nullable = true)
 |-- home_score: string (nullable = true)
 |-- away_score: string (nullable = true)
 |-- tournament: string (nullable = true)
 |-- city: string (nullable = true)
 |-- country: string (nullable = true)
 |-- neutral: boolean (nullable = true)



In [8]:
df.createOrReplaceTempView("matches")


In [9]:
spark.sql("""
SELECT COUNT(*) AS total_matches
FROM matches
""").show()


+-------------+
|total_matches|
+-------------+
|        44152|
+-------------+



In [10]:
spark.sql("""
SELECT
  MIN(YEAR(date)) AS first_year,
  MAX(YEAR(date)) AS last_year
FROM matches
""").show()


+----------+---------+
|first_year|last_year|
+----------+---------+
|      1872|     2022|
+----------+---------+



In [11]:
spark.sql("""
SELECT tournament, COUNT(*) AS matches_count
FROM matches
GROUP BY tournament
ORDER BY matches_count DESC
LIMIT 10
""").show(truncate=False)


+------------------------------------+-------------+
|tournament                          |matches_count|
+------------------------------------+-------------+
|Friendly                            |17461        |
|FIFA World Cup qualification        |7774         |
|UEFA Euro qualification             |2593         |
|African Cup of Nations qualification|1932         |
|FIFA World Cup                      |948          |
|Copa América                        |841          |
|AFC Asian Cup qualification         |764          |
|African Cup of Nations              |742          |
|CECAFA Cup                          |620          |
|CFU Caribbean Cup qualification     |606          |
+------------------------------------+-------------+



In [12]:
spark.sql("""
SELECT COUNT(*) AS neutral_matches
FROM matches
WHERE neutral = true
""").show()


+---------------+
|neutral_matches|
+---------------+
|          10996|
+---------------+



In [13]:
spark.sql("""
SELECT country, COUNT(*) AS matches_hosted
FROM matches
GROUP BY country
ORDER BY matches_hosted DESC
LIMIT 10
""").show(truncate=False)


+--------------------+--------------+
|country             |matches_hosted|
+--------------------+--------------+
|United States       |1259          |
|France              |830           |
|Malaysia            |752           |
|England             |722           |
|Sweden              |659           |
|Qatar               |618           |
|Germany             |608           |
|Brazil              |569           |
|Spain               |568           |
|United Arab Emirates|548           |
+--------------------+--------------+



In [14]:
spark.sql("""
SELECT COUNT(*) AS draws
FROM matches
WHERE home_score = away_score
""").show()


+-----+
|draws|
+-----+
|10213|
+-----+



In [15]:
spark.sql("""
SELECT *
FROM matches
WHERE home_score + away_score > 6
""").show(truncate=False)


+----------+----------------+----------------+----------+----------+--------------------+---------+--------+-------+
|date      |home_team       |away_team       |home_score|away_score|tournament          |city     |country |neutral|
+----------+----------------+----------------+----------+----------+--------------------+---------+--------+-------+
|1878-03-02|Scotland        |England         |7         |2         |Friendly            |Glasgow  |Scotland|false  |
|1878-03-23|Scotland        |Wales           |9         |0         |Friendly            |Glasgow  |Scotland|false  |
|1879-04-05|England         |Scotland        |5         |4         |Friendly            |London   |England |false  |
|1880-03-13|Scotland        |England         |5         |4         |Friendly            |Glasgow  |Scotland|false  |
|1881-03-12|England         |Scotland        |1         |6         |Friendly            |London   |England |false  |
|1882-02-18|Northern Ireland|England         |0         |13     

In [16]:
spark.sql("""
SELECT team, COUNT(*) AS total_matches
FROM (
  SELECT home_team AS team FROM matches
  UNION ALL
  SELECT away_team AS team FROM matches
)
GROUP BY team
ORDER BY total_matches DESC
""").show(truncate=False)


+-----------+-------------+
|team       |total_matches|
+-----------+-------------+
|Sweden     |1052         |
|England    |1047         |
|Brazil     |1019         |
|Argentina  |1014         |
|Germany    |986          |
|Hungary    |964          |
|Mexico     |926          |
|Uruguay    |919          |
|South Korea|904          |
|France     |875          |
|Poland     |851          |
|Italy      |837          |
|Switzerland|834          |
|Norway     |832          |
|Denmark    |832          |
|Austria    |820          |
|Netherlands|820          |
|Scotland   |814          |
|Belgium    |804          |
|Chile      |793          |
+-----------+-------------+
only showing top 20 rows



In [17]:
spark.sql("""
SELECT team, SUM(goals) AS total_goals
FROM (
  SELECT home_team AS team, home_score AS goals FROM matches
  UNION ALL
  SELECT away_team AS team, away_score AS goals FROM matches
)
GROUP BY team
ORDER BY total_goals DESC
LIMIT 10
""").show(truncate=False)


+-----------+-----------+
|team       |total_goals|
+-----------+-----------+
|England    |2282.0     |
|Brazil     |2228.0     |
|Germany    |2205.0     |
|Sweden     |2060.0     |
|Hungary    |1948.0     |
|Argentina  |1880.0     |
|Netherlands|1694.0     |
|Mexico     |1592.0     |
|South Korea|1576.0     |
|France     |1560.0     |
+-----------+-----------+



In [18]:
spark.sql("""
SELECT
  FLOOR(YEAR(date)/10)*10 AS decade,
  AVG(home_score + away_score) AS avg_goals
FROM matches
GROUP BY FLOOR(YEAR(date)/10)*10
ORDER BY decade
""").show()


+------+------------------+
|decade|         avg_goals|
+------+------------------+
|  1870| 4.538461538461538|
|  1880| 5.581818181818182|
|  1890|5.1525423728813555|
|  1900| 4.169354838709677|
|  1910| 4.007067137809187|
|  1920|3.8367626886145403|
|  1930|  4.25940594059406|
|  1940| 4.294776119402985|
|  1950| 3.980964467005076|
|  1960|3.4683274021352313|
|  1970| 2.955379908210097|
|  1980|2.4950990615224193|
|  1990| 2.730442692540934|
|  2000| 2.792825302483549|
|  2010|  2.69994863893169|
|  2020| 2.600644864117918|
+------+------------------+



In [19]:
spark.sql("""
SELECT
  tournament,
  YEAR(date) AS year,
  COUNT(*) AS matches_count
FROM matches
GROUP BY tournament, YEAR(date)
ORDER BY year, matches_count DESC
""").show(truncate=False)


+--------------------+----+-------------+
|tournament          |year|matches_count|
+--------------------+----+-------------+
|Friendly            |1872|1            |
|Friendly            |1873|1            |
|Friendly            |1874|1            |
|Friendly            |1875|1            |
|Friendly            |1876|2            |
|Friendly            |1877|2            |
|Friendly            |1878|2            |
|Friendly            |1879|3            |
|Friendly            |1880|3            |
|Friendly            |1881|3            |
|Friendly            |1882|5            |
|Friendly            |1883|5            |
|British Championship|1884|6            |
|British Championship|1885|6            |
|Friendly            |1885|1            |
|British Championship|1886|6            |
|Friendly            |1886|1            |
|British Championship|1887|6            |
|British Championship|1888|6            |
|Friendly            |1888|1            |
+--------------------+----+-------

In [20]:
spark.sql("""
SELECT home_team AS team,
       COUNT(*) AS home_wins
FROM matches
WHERE home_score > away_score
GROUP BY home_team
ORDER BY home_wins DESC
""").show(truncate=False)


+-------------+---------+
|team         |home_wins|
+-------------+---------+
|Brazil       |423      |
|Argentina    |373      |
|Germany      |327      |
|England      |323      |
|Mexico       |320      |
|South Korea  |296      |
|Sweden       |294      |
|Italy        |291      |
|France       |290      |
|Hungary      |265      |
|Egypt        |260      |
|Spain        |256      |
|Netherlands  |252      |
|United States|243      |
|Belgium      |229      |
|Denmark      |229      |
|Scotland     |224      |
|Saudi Arabia |222      |
|Austria      |220      |
|Chile        |213      |
+-------------+---------+
only showing top 20 rows



In [21]:
spark.sql("""
SELECT team,
       SUM(win)  AS wins,
       SUM(loss) AS losses,
       SUM(draw) AS draws
FROM (
  SELECT home_team AS team,
         CASE WHEN home_score > away_score THEN 1 ELSE 0 END AS win,
         CASE WHEN home_score < away_score THEN 1 ELSE 0 END AS loss,
         CASE WHEN home_score = away_score THEN 1 ELSE 0 END AS draw
  FROM matches

  UNION ALL

  SELECT away_team AS team,
         CASE WHEN away_score > home_score THEN 1 ELSE 0 END AS win,
         CASE WHEN away_score < home_score THEN 1 ELSE 0 END AS loss,
         CASE WHEN away_score = home_score THEN 1 ELSE 0 END AS draw
  FROM matches
)
GROUP BY team
ORDER BY wins DESC
""").show(truncate=False)


+-----------+----+------+-----+
|team       |wins|losses|draws|
+-----------+----+------+-----+
|Brazil     |651 |160   |208  |
|England    |593 |201   |253  |
|Germany    |573 |205   |208  |
|Argentina  |547 |213   |254  |
|Sweden     |516 |309   |227  |
|South Korea|477 |192   |235  |
|Mexico     |467 |243   |216  |
|Hungary    |452 |301   |211  |
|Italy      |445 |157   |235  |
|France     |436 |249   |190  |
|Spain      |424 |134   |173  |
|Netherlands|419 |213   |188  |
|Uruguay    |399 |294   |226  |
|Scotland   |385 |253   |176  |
|Denmark    |378 |278   |176  |
|Russia     |365 |164   |187  |
|Poland     |365 |268   |218  |
|Belgium    |354 |277   |173  |
|Zambia     |346 |219   |201  |
|Austria    |340 |303   |177  |
+-----------+----+------+-----+
only showing top 20 rows



In [22]:
spark.sql("""
SELECT
  neutral,
  AVG(home_score + away_score) AS avg_goals
FROM matches
GROUP BY neutral
""").show()


+-------+------------------+
|neutral|         avg_goals|
+-------+------------------+
|   true|3.0150671171582504|
|  false|2.8861339848580823|
+-------+------------------+



In [23]:
spark.sql("""
SELECT *,
       ABS(home_score - away_score) AS goal_diff
FROM matches
ORDER BY goal_diff DESC
LIMIT 5
""").show(truncate=False)


+----------+---------+--------------+----------+----------+----------------------------+-------------+----------------+-------+---------+
|date      |home_team|away_team     |home_score|away_score|tournament                  |city         |country         |neutral|goal_diff|
+----------+---------+--------------+----------+----------+----------------------------+-------------+----------------+-------+---------+
|2001-04-11|Australia|American Samoa|31        |0         |FIFA World Cup qualification|Coffs Harbour|Australia       |false  |31.0     |
|1971-09-13|Tahiti   |Cook Islands  |30        |0         |South Pacific Games         |Papeete      |French Polynesia|false  |30.0     |
|1979-08-30|Fiji     |Kiribati      |24        |0         |South Pacific Games         |Nausori      |Fiji            |false  |24.0     |
|2001-04-09|Australia|Tonga         |22        |0         |FIFA World Cup qualification|Coffs Harbour|Australia       |false  |22.0     |
|1966-04-03|Libya    |Oman        

In [24]:
spark.sql("""
SELECT team,
       SUM(goals_for) AS goals_for,
       SUM(goals_against) AS goals_against,
       SUM(goals_for - goals_against) AS goal_average
FROM (
  SELECT home_team AS team,
         home_score AS goals_for,
         away_score AS goals_against
  FROM matches

  UNION ALL

  SELECT away_team AS team,
         away_score AS goals_for,
         home_score AS goals_against
  FROM matches
)
GROUP BY team
ORDER BY goal_average DESC
""").show(truncate=False)


+-----------+---------+-------------+------------+
|team       |goals_for|goals_against|goal_average|
+-----------+---------+-------------+------------+
|Brazil     |2228.0   |910.0        |1318.0      |
|England    |2282.0   |1014.0       |1268.0      |
|Germany    |2205.0   |1133.0       |1072.0      |
|Argentina  |1880.0   |1042.0       |838.0       |
|Spain      |1464.0   |650.0        |814.0       |
|South Korea|1576.0   |808.0        |768.0       |
|Sweden     |2060.0   |1347.0       |713.0       |
|Netherlands|1694.0   |1000.0       |694.0       |
|Italy      |1441.0   |803.0        |638.0       |
|Mexico     |1592.0   |976.0        |616.0       |
|Iran       |968.0    |412.0        |556.0       |
|Russia     |1229.0   |696.0        |533.0       |
|Hungary    |1948.0   |1429.0       |519.0       |
|Australia  |1081.0   |583.0        |498.0       |
|France     |1560.0   |1128.0       |432.0       |
|China PR   |1097.0   |674.0        |423.0       |
|Egypt      |1096.0   |680.0   

In [25]:
spark.sql("""
SELECT *,
       ROW_NUMBER() OVER(PARTITION BY year ORDER BY wins DESC) AS rank
FROM (
  SELECT YEAR(date) AS year,
         home_team AS team,
         COUNT(*) AS wins
  FROM matches
  WHERE home_score > away_score
  GROUP BY year, home_team
)
ORDER BY year, rank
""").show(truncate=False)


+----+----------------+----+----+
|year|team            |wins|rank|
+----+----------------+----+----+
|1873|England         |1   |1   |
|1874|Scotland        |1   |1   |
|1876|Scotland        |2   |1   |
|1878|Scotland        |2   |1   |
|1879|England         |2   |1   |
|1880|Scotland        |2   |1   |
|1882|Scotland        |2   |1   |
|1882|Wales           |2   |2   |
|1883|England         |2   |1   |
|1884|Scotland        |2   |1   |
|1884|Wales           |1   |2   |
|1885|Scotland        |1   |1   |
|1885|England         |1   |2   |
|1886|Scotland        |1   |1   |
|1886|Wales           |1   |2   |
|1886|United States   |1   |3   |
|1887|England         |2   |1   |
|1887|Scotland        |1   |2   |
|1887|Northern Ireland|1   |3   |
|1888|Scotland        |2   |1   |
+----+----------------+----+----+
only showing top 20 rows



In [26]:
spark.sql("""
SELECT
  FLOOR(YEAR(date)/10)*10 AS decade,
  COUNT(*) AS matches_count
FROM matches
GROUP BY FLOOR(YEAR(date)/10)*10
ORDER BY decade
""").show()


+------+-------------+
|decade|matches_count|
+------+-------------+
|  1870|           13|
|  1880|           55|
|  1890|           59|
|  1900|          124|
|  1910|          283|
|  1920|          729|
|  1930|         1010|
|  1940|          804|
|  1950|         1576|
|  1960|         2810|
|  1970|         3922|
|  1980|         4795|
|  1990|         6596|
|  2000|         9422|
|  2010|         9735|
|  2020|         2219|
+------+-------------+



In [27]:
spark.sql("""
SELECT team, year
FROM (
  SELECT team, year,
         SUM(loss) AS losses
  FROM (
    SELECT YEAR(date) AS year,
           home_team AS team,
           CASE WHEN home_score < away_score THEN 1 ELSE 0 END AS loss
    FROM matches

    UNION ALL

    SELECT YEAR(date),
           away_team,
           CASE WHEN away_score < home_score THEN 1 ELSE 0 END AS loss
    FROM matches
  )
  GROUP BY team, year
)
WHERE losses = 0
""").show(truncate=False)


+------------------+----+
|team              |year|
+------------------+----+
|Jersey            |2009|
|Guatemala         |2020|
|Scotland          |1890|
|Kenya             |1953|
|Egypt             |1958|
|Gabon             |2007|
|Dominican Republic|2017|
|Curaçao           |1934|
|Sweden            |1940|
|German DR         |1955|
|Haiti             |1971|
|Haiti             |1979|
|Catalonia         |2003|
|Mexico            |1949|
|Romania           |1943|
|Italy             |1946|
|Guinea-Bissau     |1987|
|Jersey            |1996|
|Jordan            |1965|
|Suriname          |1967|
+------------------+----+
only showing top 20 rows



In [28]:
spark.sql("""
WITH results AS (
  SELECT date,
         home_team AS team,
         CASE WHEN home_score > away_score THEN 1 ELSE 0 END AS win
  FROM matches
  UNION ALL
  SELECT date,
         away_team AS team,
         CASE WHEN away_score > home_score THEN 1 ELSE 0 END AS win
  FROM matches
),
streaks AS (
  SELECT *,
         SUM(CASE WHEN win = 0 THEN 1 ELSE 0 END)
           OVER(PARTITION BY team ORDER BY date) AS grp
  FROM results
)
SELECT team,
       COUNT(*) AS longest_win_streak
FROM streaks
WHERE win = 1
GROUP BY team, grp
ORDER BY longest_win_streak DESC
""").show(truncate=False)


+---------+------------------+
|team     |longest_win_streak|
+---------+------------------+
|Padania  |15                |
|Spain    |15                |
|Brazil   |14                |
|Brazil   |14                |
|France   |14                |
|Brazil   |13                |
|Australia|13                |
|Guyana   |13                |
|Mexico   |13                |
|Italy    |13                |
|Scotland |13                |
|Belgium  |12                |
|France   |12                |
|German DR|12                |
|Germany  |12                |
|Indonesia|12                |
|Morocco  |12                |
|Russia   |12                |
|Spain    |12                |
|Australia|11                |
+---------+------------------+
only showing top 20 rows



In [29]:
spark.sql("""
WITH wins AS (
  SELECT tournament, home_team AS team
  FROM matches
  WHERE home_score > away_score
  UNION ALL
  SELECT tournament, away_team AS team
  FROM matches
  WHERE away_score > home_score
)
SELECT tournament, team, wins_count
FROM (
  SELECT tournament, team,
         COUNT(*) AS wins_count,
         ROW_NUMBER() OVER(PARTITION BY tournament ORDER BY COUNT(*) DESC) AS r
  FROM wins
  GROUP BY tournament, team
)
WHERE r = 1
""").show(truncate=False)


+-------------------------------------------+------------------+----------+
|tournament                                 |team              |wins_count|
+-------------------------------------------+------------------+----------+
|ABCS Tournament                            |Suriname          |7         |
|AFC Asian Cup                              |Iran              |41        |
|AFC Asian Cup qualification                |Iran              |36        |
|AFC Challenge Cup                          |Tajikistan        |11        |
|AFC Challenge Cup qualification            |Philippines       |7         |
|AFF Championship                           |Thailand          |42        |
|AFF Championship qualification             |Timor-Leste       |2         |
|African Cup of Nations                     |Egypt             |60        |
|African Cup of Nations qualification       |Ivory Coast       |67        |
|African Nations Championship               |Morocco           |11        |
|African Nat

In [30]:
spark.sql("""
SELECT team,
       SUM(home_matches) AS home_matches,
       SUM(away_matches) AS away_matches,
       SUM(home_wins) AS home_wins,
       SUM(away_wins) AS away_wins
FROM (
  SELECT home_team AS team,
         1 AS home_matches, 0 AS away_matches,
         CASE WHEN home_score > away_score THEN 1 ELSE 0 END AS home_wins,
         0 AS away_wins
  FROM matches

  UNION ALL

  SELECT away_team,
         0, 1,
         0,
         CASE WHEN away_score > home_score THEN 1 ELSE 0 END AS away_wins
  FROM matches
)
GROUP BY team
ORDER BY home_wins DESC
""").show(truncate=False)


+-------------+------------+------------+---------+---------+
|team         |home_matches|away_matches|home_wins|away_wins|
+-------------+------------+------------+---------+---------+
|Brazil       |593         |426         |423      |228      |
|Argentina    |567         |447         |373      |174      |
|Germany      |526         |460         |327      |246      |
|England      |523         |524         |323      |270      |
|Mexico       |549         |377         |320      |147      |
|South Korea  |499         |405         |296      |181      |
|Sweden       |505         |547         |294      |222      |
|Italy        |466         |371         |291      |154      |
|France       |503         |372         |290      |146      |
|Hungary      |474         |490         |265      |187      |
|Egypt        |403         |278         |260      |77       |
|Spain        |379         |352         |256      |168      |
|Netherlands  |439         |381         |252      |167      |
|United 